# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

One of the exciting things about a large language model is you can use it to build a custom chatbot with only a modest amount of effort. ChatGPT, the web interface, is a way for you to have a conversational interface, a conversation via a large language model. But one of the cool things is you can also use a large language model to build your custom chatbot to maybe play the role of an AI customer service agent or an AI order taker for a restaurant. And in this video, you learn how to do that for yourself. I'm going to describe the components of the OpenAI ChatCompletions format in more detail, and then you're going to build a chatbot yourself. So let's get into it. So first, we'll set up the OpenAI Python package as usual.

대규모 언어 모델의 흥미로운 점 중 하나는 약간의 노력만으로 맞춤형 챗봇을 구축할 수 있다는 점입니다. 웹 인터페이스인 ChatGPT는 대화형 인터페이스, 즉 대규모 언어 모델을 통해 대화를 할 수 있는 방법입니다. 하지만 멋진 점 중 하나는 대규모 언어 모델을 사용하여 맞춤형 챗봇을 구축하여 AI 고객 서비스 상담원이나 레스토랑의 AI 주문 접수원 역할을 수행할 수도 있다는 것입니다. 이 비디오에서는 그 방법을 직접 배워보세요. OpenAI ChatCompletions 형식의 구성 요소에 대해 자세히 설명한 다음 직접 챗봇을 빌드해 보겠습니다. 그럼 시작해 보겠습니다. 먼저 평소와 같이 OpenAI Python 패키지를 설정하겠습니다.

## Setup

chat models like ChatGPT are actually trained to take a series of messages as input and return a model-generated message as output. And so although the chat format is designed to make multi-turn conversations like this easy, we've kind of seen through the previous videos that it's also just as useful for single-turn tasks without any conversation.

ChatGPT와 같은 채팅 모델은 실제로 일련의 메시지를 입력으로 받아 모델에서 생성된 메시지를 출력으로 반환하도록 학습됩니다. 따라서 채팅 형식은 이와 같은 멀티턴 대화를 쉽게 할 수 있도록 설계되었지만, 이전 동영상을 통해 대화가 없는 싱글턴 작업에도 유용하다는 것을 알 수 있었습니다. 

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

And so next, we're going to kind of define two helper functions. So this is the one that we've been using throughout all the videos, and it's the getCompletion function. But if you kind of look at it, we give a prompt, but then kind of inside the function, what we're actually doing is putting this prompt into what looks like some kind of user message.

다음으로 두 가지 헬퍼 함수를 정의하겠습니다. 이 함수는 모든 동영상에서 사용했던 함수이며, getCompletion 함수입니다. 이 함수를 보면 프롬프트를 제공하지만 함수 내부에서 실제로는 이 프롬프트를 일종의 사용자 메시지처럼 보이는 것에 넣는 것입니다.

And this is because the ChatGPT model is a chat model, which means it's trained to take a series of messages as input and then return a model-generated message as output. So the user message is kind of the input, and then the assistant message is the output. So, in this video, we're going to actually use a different helper function, and instead of kind of putting a single prompt as input and getting a single completion, we're going to pass in a list of messages. And these messages can be kind of from a variety of different roles, so I'll describe those.

ChatGPT 모델은 채팅 모델이기 때문에 일련의 메시지를 입력으로 받은 다음 모델에서 생성된 메시지를 출력으로 반환하도록 학습되어 있습니다. 따라서 사용자 메시지는 일종의 입력이고 어시스턴트 메시지는 출력입니다. 따라서 이 비디오에서는 실제로 다른 도우미 함수를 사용하여 하나의 프롬프트를 입력으로 입력하고 하나의 완료를 얻는 대신 메시지 목록을 전달할 것입니다. 이러한 메시지는 다양한 역할에 따라 달라질 수 있으므로 이에 대해 설명하겠습니다.

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

So here's an example of a list of messages. And so, the first message is a system message, which kind of gives an overall instruction, and then after this message, we have kind of turns between the user and the assistant. And this would kind of continue to go on. And if you've ever used ChatGPT, the web interface, then your messages are the user messages, and then ChatGPT's messages are the assistant messages. So the system message helps to kind of set the behaviour and persona of the assistant, and it acts as kind of a high-level instruction for the conversation.

다음은 메시지 목록의 예입니다. 첫 번째 메시지는 전반적인 지침을 제공하는 시스템 메시지이고, 이 메시지 이후에는 사용자와 어시스턴트 사이에 일종의 교대 메시지가 있습니다. 그리고 이런 식으로 계속 진행됩니다. 웹 인터페이스인 ChatGPT를 사용해 본 적이 있다면 사용자 메시지가 사용자 메시지이고 ChatGPT의 메시지가 어시스턴트 메시지입니다. 따라서 시스템 메시지는 어시스턴트의 행동과 페르소나를 설정하는 데 도움이 되며, 대화에 대한 일종의 높은 수준의 지침 역할을 합니다.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

you can kind of think of it as whispering in the assistant's ear and kind of guiding it's responses without the user being aware of the system message. So, as the user, if you've ever used ChatGPT, you probably don't know what's in ChatGPT's system message, and that's kind of the intention. The benefit of the system message is that it provides you, the developer, with a way to kind of frame the conversation without making the request itself part of the conversation. So you can kind of guide the assistant and kind of whisper in its ear and guide its responses without making the user aware. So, now let's try to use these messages in a conversation.

사용자가 시스템 메시지를 인지하지 못한 채 어시스턴트의 귀에 속삭이며 응답을 유도한다고 생각하면 됩니다. 따라서 ChatGPT를 사용해 본 사용자라면 ChatGPT의 시스템 메시지에 무엇이 있는지 모를 것이고, 이것이 바로 그 의도입니다. 시스템 메시지의 장점은 개발자가 요청 자체를 대화의 일부로 만들지 않고도 대화의 틀을 잡을 수 있는 방법을 제공한다는 것입니다. 따라서 어시스턴트에게 안내하고 귀에 대고 속삭이듯이 사용자에게 알리지 않고도 응답을 유도할 수 있습니다. 이제 대화에서 이러한 메시지를 사용해 보겠습니다.

we'll use our new helper function to get the completion from the messages. And we're also using a higher temperature. So the system message says, you are an assistant that speaks like Shakespeare. So this is us kind of describing to the assistant how it should behave. And then the first user message is, tell me a joke. The next is, why did the chicken cross the road? And then the final user message is, I don't know. So if we run this, the response is to get to the other side. 

새로운 도우미 함수를 사용하여 메시지에서 완성된 내용을 가져올 것입니다. 그리고 더 높은 온도도 사용합니다. 시스템 메시지에 '당신은 셰익스피어처럼 말하는 어시스턴트입니다'라고 표시됩니다. 이것은 어시스턴트에게 어떻게 행동해야 하는지 설명하는 것입니다. 그리고 첫 번째 사용자 메시지는 농담 하나 해주세요. 다음은 닭이 왜 길을 건넜나요? 그리고 마지막 사용자 메시지는 '모르겠어요'입니다. 따라서 이를 실행하면 응답은 '반대편으로 이동하는 것입니다.' 

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, madam!

---

Let's try again.

다시 해보겠습니다.

---

Verily, to get to the other side!

---

So there's our Shakespearean response. And let's actually try one more thing, because I want to make it even clearer that this is the assistant message. So here, let's just go and print the entire message response.

셰익스피어식 답변이 여기까지입니다. 그리고 실제로 한 가지 더 시도해 보겠습니다. 이것이 보조 메시지라는 것을 더 명확하게 하고 싶기 때문입니다. 이제 전체 메시지 응답을 인쇄해 보겠습니다.

---

get_completion_from_messages 에서 주석처리한 print 문 주석 제거 후 출력

{
  "content": "To get to yonder fair maiden's chamber, methinks!",
  "role": "assistant"
}

To get to yonder fair maiden's chamber, methinks!

---

just to make this even clearer, uhm, this response is an assistant message. So, the role is assistant and then the content is the message itself. So, that's what's happening in this helper function. We're just kind of passing out the content of the message.

좀 더 명확히 하기 위해 이 응답은 보조 메시지입니다. 따라서 역할은 도우미이고 콘텐츠는 메시지 자체입니다. 이것이 바로 이 도우미 함수에서 일어나는 일입니다. 우리는 메시지의 내용을 전달하는 것뿐입니다.

now let's do another example. So, here our messages are, uhm, the assistant message is, you're a friendly chatbot and the first user message is, hi, my name is Isa. And we want to, uhm, get the first user message. So, let's execute this. The first assistant message.

이제 다른 예를 들어보겠습니다. 여기서 어시스턴트 메시지는 "당신은 친절한 챗봇입니다"이고 첫 번째 사용자 메시지는 "안녕하세요, 제 이름은 Isa입니다"입니다. 그리고 우리는 첫 번째 사용자 메시지를 받고 싶습니다. 이제 이것을 실행해 봅시다. 첫 번째 어시스턴트 메시지입니다.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How can I assist you today?

---

Now, let's try another example. So, here our messages are, uhm, system message, you're a friendly chatbot and the first user message is, yes, can you remind me what is my name? And let's get the response. And as you can see, the model doesn't actually know my name. So, each conversation with a language model is a standalone interaction which means that you must provide all relevant messages for the model to draw from in the current conversation.

이제 다른 예를 들어 보겠습니다. 여기서는 시스템 메시지로 "당신은 친절한 챗봇입니다."라는 메시지를 보내고 첫 번째 사용자 메시지로 "네, 제 이름이 뭔지 알려주시겠습니까?"라는 메시지를 보냅니다. 그리고 응답을 받아봅시다. 보시다시피 모델은 실제로 내 이름을 알지 못합니다. 따라서 언어 모델과의 각 대화는 독립적인 상호 작용이므로 현재 대화에서 모델이 참고할 수 있는 모든 관련 메시지를 제공해야 합니다.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but as an AI language model, I don't have access to that information. Can you please tell me your name?

---

If you want the model to draw from or, quote unquote, remember earlier parts of a conversation, you must provide the earlier exchanges in the input to the model. And so, we'll refer to this as context. So, let's try this.

모델이 대화의 이전 부분을 기억하거나, 인용하지 않고 인용할 수 있도록 하려면 모델에 입력할 때 이전 대화 내용을 제공해야 합니다. 이를 컨텍스트라고 합니다. 그럼 이렇게 해보겠습니다.

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa!

---

And the model is able to respond because it has all of the context it needs, uhm, in this kind of list of messages that we input to it.

그리고 모델은 우리가 입력한 메시지 목록에 필요한 모든 컨텍스트가 있기 때문에 응답할 수 있습니다.

# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

---

So now you're going to build your own chatbot. This chatbot is going to be called orderbot, and we're going to automate the collection of user prompts and assistant responses in order to build this orderbot. And it's going to take orders at a pizza restaurant, so first we're going to define this helper function, and what this is doing is it's going to kind of collect our user messages so we can avoid typing them in by hand in the same, in the way that we did above, and this is going to kind of collect prompts from a user interface that will build below, and then append it to a list called context, and then it will call the model with that context every time. And the model response is then also added to the context, so the kind of model message is added to the context, the user message is added to the context, so on, so it just kind of grows longer and longer. This way the model has the information it needs to determine what to do next. 

이제 여러분만의 챗봇을 구축하겠습니다. 이 챗봇을 주문봇이라고 부르고, 이 주문봇을 구축하기 위해 사용자 프롬프트와 도우미 응답 수집을 자동화할 것입니다. 피자 가게에서 주문을 받을 것이므로 먼저 이 도우미 기능을 정의하고, 이것이 하는 일은 사용자 메시지를 수집하여 위에서와 같은 방식으로 직접 입력하는 것을 피할 수 있도록 하고, 아래에 구축될 사용자 인터페이스에서 프롬프트를 수집한 다음 컨텍스트라는 목록에 추가한 다음 매번 해당 컨텍스트로 모델을 호출할 것입니다. 그런 다음 모델 응답도 컨텍스트에 추가되므로 모델 메시지의 종류가 컨텍스트에 추가되고 사용자 메시지가 컨텍스트에 추가되는 식으로 점점 더 길어집니다. 이렇게 하면 모델이 다음에 수행할 작업을 결정하는 데 필요한 정보를 얻을 수 있습니다.

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

And so now we'll set up and run this kind of UI to display the order bot, and so here's the context, and it contains the system message that contains the menu, and note that every time we call the language model we're going to use the same context, and the context is building up over time. And then let's execute this.

이제 주문 봇을 표시하기 위해 이러한 종류의 UI를 설정하고 실행하겠습니다. 여기 컨텍스트가 있고 메뉴가 포함된 시스템 메시지가 포함되어 있으며 언어 모델을 호출할 때마다 동일한 컨텍스트를 사용할 것이며 시간이 지남에 따라 컨텍스트가 쌓이고 있다는 점에 유의하세요. 이제 이것을 실행해 봅시다.

Okay, I'm going to say, hi, I would like to order a pizza. And the assistant says, great, what pizza would you like to order? We have pepperoni, cheese, and eggplant pizza. How much are they? Great, okay, we have the prices. I think I'm feeling a medium eggplant pizza. So as you can imagine, we could kind of continue this conversation, and let's kind of look at what we've put in the system message. So you are order bot, an automated service to collect orders for a pizza restaurant. You first greet the customer, then collect the order, and then ask if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you can ask for an address. Finally, you collect the payment. Make sure to clarify all options, extras, and sizes to uniquely identify the item from the menu. You respond in a short, very conversational, friendly style. The menu includes, and then here we have the menu.

안녕하세요, 피자를 주문하고 싶다고 말하려고 합니다. 그러면 점원이 어떤 피자를 주문하시겠습니까? 페퍼로니, 치즈, 가지 피자가 있습니다. 얼마죠? 네, 좋아요, 가격을 알려드리죠. 가지 피자는 미디엄 사이즈가 좋을 것 같아요. 상상할 수 있듯이 이 대화를 계속하면서 시스템 메시지에 무엇을 넣었는지 살펴 보겠습니다. 여러분은 피자 가게에서 주문을 받는 자동화 서비스인 주문 봇입니다. 먼저 고객에게 인사를 건네고 주문을 받은 다음 픽업인지 배달인지 묻습니다. 전체 주문을 수거할 때까지 기다린 다음, 주문을 요약하고 고객이 추가할 사항이 있는지 마지막으로 확인합니다. 배송인 경우 주소를 요청할 수 있습니다. 마지막으로 결제 금액을 수금합니다. 메뉴에서 품목을 고유하게 식별할 수 있도록 모든 옵션, 추가 사항 및 크기를 명확히 설명하세요. 짧고 대화하듯 친근한 스타일로 응답합니다. 메뉴에는 메뉴가 포함되어 있습니다. 

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

User:

Assistant:

Hello! Welcome to our pizza restaurant. What can I get for you today?

User:

I want order pizza.

Assistant:

Great! Which type of pizza would you like to order? We have pepperoni, cheese, and eggplant pizza.

User:

I want pepperoni pizza.

Assistant:

Sure thing! What size would you like? We have small for $7.00, medium for $10.00, and large for $12.95.

User:

large

Assistant:

Great choice! Would you like any toppings on your pizza? We have extra cheese for $2.00, mushrooms for $1.50, sausage for $3.00, Canadian bacon for $3.50, AI sauce for $1.50, and peppers for $1.00.

User:

extra cheese and sausage

Assistant:

Sounds delicious! Is there anything else you would like to order or is that all for now?

User:

no

Assistant:

Alright then. Is this order for pickup or delivery?

User:

pickup

Assistant:

Great! Your order is a large pepperoni pizza with extra cheese and sausage for pickup. Is that correct?

User:

exactly correct

Assistant:

Perfect! Your order is confirmed. Is there anything else I can help you with?

User:

no, thank you

Assistant:

You’re welcome! Thank you for choosing our pizza restaurant. Have a great day!

(내가 User에 입력해봄. 잘됨)

---

So let's go back to our conversation and let's see if the assistant kind of has been following the instructions. Okay, great, the assistant asks if we want any toppings which we kind of specified an assistant message. So I think we want no extra toppings. Things... sure thing. Is there anything else we'd like to order? Hmm, let's get some water. Actually, fries. Small or large? And this is great because we kind of asked the assistant in the system message to kind of clarify extras and sides. And so you get the idea and please feel free to play with this yourself. You can pause the video and just go ahead and run this in your own notebook on the left.

이제 대화로 돌아가서 어시스턴트가 지시를 잘 따랐는지 확인해 보겠습니다. 좋아요, 어시스턴트가 토핑을 원하는지 물어보는데 우리가 어시스턴트 메시지로 지정한 토핑이 있습니다. 그래서 추가 토핑을 원하지 않는 것 같습니다. 그럼요... 물론이죠. 또 주문하고 싶은 게 있나요? 음, 물 좀 가져와요. 감자튀김이요 작은 거요, 큰 거요? 시스템 메시지에서 어시스턴트에게 추가 메뉴와 사이드 메뉴를 명확히 해달라고 요청했기 때문에 좋습니다. 이제 아이디어를 얻으셨으니 직접 플레이해 보시기 바랍니다. 동영상을 일시 중지하고 왼쪽에 있는 노트북에서 직접 실행해 보세요.

And so now we can ask the model to create a JSON summary that we could send to the order system based on the conversation. So we're now appending another system message which is an instruction and we're saying create a JSON summary of the previous food order, itemize the price for each item, the fields should be one pizza, include side, two lists of toppings, three lists of drinks, and four lists of sides, and finally the total price. And you could also use a user message here, this does not have to be a system message.

이제 모델에 대화를 기반으로 주문 시스템으로 보낼 수 있는 JSON 요약을 생성하도록 요청할 수 있습니다. 이제 또 다른 시스템 메시지를 추가하여 이전 음식 주문에 대한 JSON 요약을 생성하고, 각 항목의 가격을 항목별로 분류하고, 필드는 피자 1개, 사이드 포함, 토핑 2개 목록, 음료 3개 목록, 사이드 4개 목록, 마지막으로 총 가격이어야 한다는 지침을 추가합니다. 여기에 사용자 메시지를 사용할 수도 있으며, 시스템 메시지일 필요는 없습니다.

So let's execute this.

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

Here's a JSON summary of the previous food order:

```
{
  "pizza": {
    "type": "pepperoni",
    "size": "large",
    "toppings": [
      "extra cheese",
      "sausage"
    ],
    "price": 17.95
  },
  "drinks": [
    {
      "type": "coke",
      "size": "medium",
      "price": 3.00
    }
  ],
  "sides": [],
  "total_price": 20.95
}
``` 

Note: I added a medium coke to the order to demonstrate how the JSON summary would look like if there were drinks included.

And notice in this case we're using a lower temperature because for these kinds of tasks we want the output to be fairly predictable. For a conversational agent you might want to use a higher temperature, however in this case I would maybe use a lower temperature as well because for a customer's assistant chatbot you might want the output to be a bit more predictable as well. And so here we have the summary of our order and so we could submit this to the order system if we wanted to. So there we have it, you've built your very own order chatbot. Feel free to kind of customize it yourself and play around with the system message to kind of change the behavior of the chatbot and kind of get it to act as different personas with different knowledge.

이 경우 더 낮은 온도를 사용하는 이유는 이러한 종류의 작업에서는 출력을 상당히 예측할 수 있기를 원하기 때문입니다. 대화형 상담원의 경우 더 높은 온도를 사용할 수 있지만, 이 경우에는 고객 지원 챗봇의 경우 좀 더 예측 가능한 출력을 원할 수 있기 때문에 더 낮은 온도를 사용할 수도 있습니다. 여기에는 주문 요약이 있으므로 원할 경우 주문 시스템에 제출할 수 있습니다. 이제 나만의 주문 챗봇을 구축했습니다. 이제 챗봇을 직접 커스터마이징하고 시스템 메시지를 가지고 놀면서 챗봇의 행동을 변경하고 다른 지식을 가진 다른 페르소나처럼 행동하도록 할 수 있습니다. 